**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity 
BASE DIRECTORY (FD_BASE): /mount 
REPO DIRECTORY (FD_REPO): /mount/repo 
WORK DIRECTORY (FD_WORK): /mount/work 
DATA DIRECTORY (FD_DATA): /mount/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /mount/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /mount/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /mount/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /mount/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /mount/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /mount/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /mount/repo/Proj_ENCODE_FCC/log 
PROJECT APP     (FD_APP): /mount/repo/Proj_ENCODE_FCC/app 
PROJECT REF     (FD_REF): /mount/repo/Proj_ENCODE_FCC/references 



In [2]:
txt_region = "fcc_astarr_macs_input_overlap"
txt_fdiry  = file.path(FD_RES, "region_coverage_fcc", txt_region, "summary")
txt_fname  = "result.coverage.zscore.final.tsv"
txt_fpath  = file.path(txt_fdiry, txt_fname)

dat = read_tsv(txt_fpath, show_col_types = FALSE)
dat_region_score = dat

print(dim(dat))
fun_display_table(head(dat, 3))

[1] 432505      9


Chrom,ChromStart,ChromEnd,Region,Score,Assay_Name,Assay_Type,Assay_Group,Assay_Label
chr10,100729094,100729750,chr10:100729094-100729750,-0.3065107,CRISPRi_FlowFISH_K562_Riley_JinWoo,CRISPRi-HCRFF,CRISPRi-HCRFF,CRISPRi-HCR FlowFISH
chr10,100743501,100744571,chr10:100743501-100744571,-0.2702473,CRISPRi_FlowFISH_K562_Riley_JinWoo,CRISPRi-HCRFF,CRISPRi-HCRFF,CRISPRi-HCR FlowFISH
chr10,100745413,100745741,chr10:100745413-100745741,0.1130381,CRISPRi_FlowFISH_K562_Riley_JinWoo,CRISPRi-HCRFF,CRISPRi-HCRFF,CRISPRi-HCR FlowFISH


In [6]:
dat = dat_region_score
table(dat$Assay_Type)


        ASTARR CRISPRi-Growth  CRISPRi-HCRFF          LMPRA          TMPRA 
        150040          72743            941          61478            823 
        WSTARR 
        146480 

In [5]:
dat = dat_region_score
table(dat$Assay_Group)


   ASTARR_KS91 CRISPRi-Growth  CRISPRi-HCRFF          LMPRA          TMPRA 
        150040          72743            941          61478            823 
        WSTARR 
        146480 

In [7]:
dat = dat_region_score
vec = c("ASTARR", "WSTARR", "LMPRA")
dat = dat %>% dplyr::filter(Assay_Type %in% vec) %>% dplyr::mutate(Assay = Assay_Type)

dat = dat %>%
    dplyr::select(Assay_Type, Region, Score) %>%
    tidyr::spread(Assay_Type, Score) %>%
    na.omit

### assign and show
mat_region_score = dat
print(dim(dat))
head(dat)

[1] 60618     4


Region,ASTARR,LMPRA,WSTARR
<chr>,<dbl>,<dbl>,<dbl>
chr1:100036895-100039189,1.71301173,1.832751,2.22355898
chr1:100048290-100048918,-1.20269450,-1.732575,-1.47308866
chr1:100050225-100050529,0.91339185,-1.159651,1.04945863
chr1:100132375-100133491,2.93068403,0.791642,2.53316751
chr1:10015258-10016191,-0.08094399,-1.285338,-0.95298857
chr1:100248956-100250290,1.13654552,1.871504,0.03547326


In [9]:
fun_ranknorm = function(x){
    return(rank(x)/length(x))
}

In [11]:
dat = mat_region_score
dat = dat %>% 
    dplyr::mutate(across(2:4, fun_ranknorm)) %>% 
    dplyr::rowwise() %>%
    dplyr::mutate(Mean = mean(c(ASTARR, WSTARR, LMPRA))) %>%
    dplyr::ungroup()

mat_region_ranknorm = dat
head(dat)

Region,ASTARR,LMPRA,WSTARR,Mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1:100036895-100039189,0.90324656,0.93528325,0.96270085,0.93374355
chr1:100048290-100048918,0.04957273,0.01337062,0.03498961,0.03264432
chr1:100050225-100050529,0.74992576,0.07186809,0.80123066,0.54100817
chr1:100132375-100133491,0.98051734,0.79668580,0.97723448,0.91814588
chr1:10015258-10016191,0.34964862,0.04888812,0.10084463,0.16646046
chr1:100248956-100250290,0.80720248,0.93898677,0.44178297,0.72932407


In [13]:
txt_region = "fcc_astarr_macs_input_overlap"
txt_fdiry  = file.path(FD_RES, "region_integration", txt_region)
txt_fname  = "result.coverage.ranknorm.whg.tsv"
txt_fpath  = file.path(txt_fdiry, txt_fname)

dat = mat_region_ranknorm
write_tsv(dat, txt_fpath)